1. В коде из методички реализуйте один или несколько из критериев останова (количество листьев, количество используемых признаков, глубина дерева и т.д.).
2. Для задачи классификации обучить дерево решений с использованием критериев разбиения Джини и Энтропия. Сравнить качество классификации, сделать выводы.
3. [опция]. Реализуйте дерево для задачи регрессии. Возьмите за основу дерево, реализованное в методичке,
    заменив механизм предсказания в листе на взятие среднего значения по выборке, и критерий Джини на дисперсию значений.

In [1]:
import matplotlib.pyplot as plt
import random

from matplotlib.colors import ListedColormap
from sklearn import datasets
from sklearn.model_selection import train_test_split

import numpy as np

In [13]:
class Node:

    def __init__(self, index, border, true_branch, false_branch):
        self.index = index  
        self.border = border
        self.true_branch = true_branch
        self.false_branch = false_branch


class Leaf:

    def __init__(self, data, labels):
        self.data = np.array(data)
        self.labels = np.array(labels)
        self.prediction = self.predict()
        
    def predict(self):
        classes, labels_cnt = np.unique(self.labels, return_counts=True)
        prediction = classes[labels_cnt == labels_cnt.max()][0]
        return prediction
    
class MyTree:
    
    def __init__(self,
                 max_depth: int=None,
                 min_leaves: int=1,
                 max_leaves: int=None,
                 gini: bool=True,
                 entropy: bool=False
                 ):
        self.max_depth = max_depth # N nodes on tree
        self.min_leaves = max_leaves # at least N leaves on tree
        self.max_leaves = max_leaves
        self.gini = gini
        self.entropy = entropy
        self.n_leaves = 0
        
    def make_split(self, data, labels, index, t):
        left = np.where(data[:, index] <= t)
        right = np.where(data[:, index] > t)
        
        true_data = data[left]
        false_data = data[right]
        true_labels = labels[left]
        false_labels = labels[right]
        return true_data, false_data, true_labels, false_labels
    
    def get_quality(self, left_labels, right_labels, base_crit):
        p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])
        if self.gini:
            _criterion = self.get_gini
        else:
            _criterion = self.get_entropy
        return base_crit - p * _criterion(left_labels) - (1 - p) * _criterion(right_labels)
        
    def get_gini(self, labels: np.array):       
        labels = np.array(labels)
        classes, size = np.unique(labels, return_counts=True)
        impurity = 1 - ((size / labels.shape) ** 2).sum()
        return impurity
        
    def get_entropy(self, labels: np.array):
        labels = np.array(labels)
        classes, size = np.unique(labels, return_counts=True)
        p = size / labels.shape
        impurity = - (p * np.log2(p)).sum()
        return impurity
    
    def fit(self, data, labels):
        self.tree = self.build_tree(data, labels)
    
    def build_tree(self, data, labels, depth=0):
        quality, t, index = self.find_best_split(data, labels)

        if quality == 0 or (self.max_depth is not None and depth >= self.max_depth)\
        or (self.max_leaves is not None and self.n_leaves >= self.max_leaves - 1):
            self.n_leaves += 1
            return Leaf(data, labels)

        true_data, false_data, true_labels, false_labels = self.make_split(data, labels, index, t)        
        true_branch = self.build_tree(true_data, true_labels, depth + 1)
        false_branch = self.build_tree(false_data, false_labels, depth + 1)

        return Node(index, t, true_branch, false_branch)
            
    def find_best_split(self, data, labels):
        if self.gini:
            criterion = self.get_gini(labels)
        else:
            criterion = self.get_entropy(labels)
        best_quality = 0
        best_t = None
        best_index = None
        n_features = data.shape[1]

        for index in range(n_features):
            t_values = np.unique(data[:, index])
            for t in t_values:
                true_data, false_data, true_labels, false_labels = self.make_split(data, labels, index, t)
                if min(len(true_data), len(false_data)) < self.min_leaves:
                    continue

                current_quality = self.get_quality(true_labels, false_labels, criterion)

                if current_quality > best_quality:
                    best_quality, best_t, best_index = current_quality, t, index

        return best_quality, best_t, best_index
        
    def find_leves(self, example, node):
        if isinstance(node, Leaf):
            return node.prediction            

        if example[node.index] <= node.border:
            return self.find_leves(example, node.true_branch)
        
        else:
            return self.find_leves(example, node.false_branch)
    
    def predict(self, X):        
        predictions = []
        for example in X:
            prediction = self.find_leves(example, self.tree)
            predictions.append(prediction)
        return predictions    

In [3]:
def accuracy(actual, predicted):
    return (actual == predicted).sum() / actual.shape[0]

In [4]:
data, labels = datasets.make_classification(
    n_features = 3,
    n_informative = 2, 
    n_classes = 2,
    n_redundant=0,
    n_clusters_per_class=1,
    random_state=5
)

In [5]:
# Для задачи классификации обучить дерево решений с использованием критериев разбиения Джини и Энтропия. Сравнить качество классификации, сделать выводы.

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.3)
tree_gini = MyTree(max_depth=5, max_leaves=6, gini=True, entropy=False)
tree_gini.fit(X_train, y_train)
tree_entropy = MyTree(max_depth=5, max_leaves=6, gini=False, entropy=True)
tree_entropy.fit(X_train, y_train)

In [6]:
pred_gini_train = np.array(tree_gini.predict(X_train))

In [7]:
pred_ent_train = np.array(tree_entropy.predict(X_train))

In [8]:
pred_gini_test = np.array(tree_gini.predict(X_test))

In [9]:
pred_ent_test = np.array(tree_entropy.predict(X_test))

In [10]:
pred_gini_train == pred_ent_train

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True])

In [11]:
pred_gini_test == pred_ent_test

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

Как видно, критерии информативности Джини и энтропии сработали одинаково.

In [12]:
print(f'\t\tgini:\t\tentropy:\ntrain_accuracy: {accuracy(y_train, pred_gini_train):.4f}\t\t{accuracy(y_train, pred_ent_train):.4f}\ntest_accuracy:  {accuracy(y_test, pred_gini_test):.4f}\t\t{accuracy(y_test, pred_ent_test):.4f}')

		gini:		entropy:
train_accuracy: 0.9571		0.9571
test_accuracy:  0.9000		0.9000
